# LSTM tutorial

Pytorch’s LSTM expects all of its inputs to be 3D tensors.
The semantics of the axes of these tensors is important:

* The first axis is the sequence itself,
* the second indexes instances in the mini-batch,
* and the third indexes elements of the input.

In [1]:
# Author: Robert Guthrie

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [54]:
inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5
inputs

[tensor([[ 0.8123,  0.4575, -0.9634]]),
 tensor([[ 0.3268, -0.0092,  0.8424]]),
 tensor([[-0.2896, -0.2440, -0.5013]]),
 tensor([[-0.4043,  0.1324,  0.2618]]),
 tensor([[ 0.7479, -0.7517, -0.0047]])]

In [3]:
lstm = nn.LSTM(3, 3)  # Input dim is 3, output dim is 3

In [5]:
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))
hidden

(tensor([[[-0.1473,  0.6272,  1.0935]]]),
 tensor([[[ 0.0939,  1.2381, -1.3459]]]))

In [6]:
for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)
    print(f"Out: {out}")
    print(f"Hidden: {hidden}")
    print("")

Out: tensor([[[ 0.1308,  0.2317, -0.2969]]], grad_fn=<MkldnnRnnLayerBackward0>)
Hidden: (tensor([[[ 0.1308,  0.2317, -0.2969]]], grad_fn=<StackBackward0>), tensor([[[ 0.3348,  0.4114, -0.4703]]], grad_fn=<StackBackward0>))

Out: tensor([[[0.1394, 0.0379, 0.0920]]], grad_fn=<MkldnnRnnLayerBackward0>)
Hidden: (tensor([[[0.1394, 0.0379, 0.0920]]], grad_fn=<StackBackward0>), tensor([[[0.3892, 0.1117, 0.1668]]], grad_fn=<StackBackward0>))

Out: tensor([[[0.0924, 0.0392, 0.1085]]], grad_fn=<MkldnnRnnLayerBackward0>)
Hidden: (tensor([[[0.0924, 0.0392, 0.1085]]], grad_fn=<StackBackward0>), tensor([[[0.5721, 0.1021, 0.1846]]], grad_fn=<StackBackward0>))

Out: tensor([[[ 0.2269, -0.0261,  0.3060]]], grad_fn=<MkldnnRnnLayerBackward0>)
Hidden: (tensor([[[ 0.2269, -0.0261,  0.3060]]], grad_fn=<StackBackward0>), tensor([[[ 0.6940, -0.1030,  0.5651]]], grad_fn=<StackBackward0>))

Out: tensor([[[ 0.2645, -0.0388,  0.4247]]], grad_fn=<MkldnnRnnLayerBackward0>)
Hidden: (tensor([[[ 0.2645, -0.0388,  0.42

In [7]:
# alternatively, we can do the entire sequence all at once.
# the first value returned by LSTM is all of the hidden states throughout
# the sequence. the second is just the most recent hidden state
# (compare the last slice of "out" with "hidden" below, they are the same)
# The reason for this is that:
# "out" will give you access to all hidden states in the sequence
# "hidden" will allow you to continue the sequence and backpropagate,
# by passing it as an argument  to the lstm at a later time
# Add the extra 2nd dimension

inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # clean out hidden state
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)

tensor([[[-0.1521, -0.2757,  0.3132]],

        [[ 0.0221, -0.2270,  0.3737]],

        [[ 0.0728, -0.2071,  0.2309]],

        [[ 0.2170, -0.1141,  0.3391]],

        [[ 0.2636, -0.0775,  0.4328]]], grad_fn=<MkldnnRnnLayerBackward0>)
(tensor([[[ 0.2636, -0.0775,  0.4328]]], grad_fn=<StackBackward0>), tensor([[[ 0.7047, -0.3022,  0.9152]]], grad_fn=<StackBackward0>))


In [55]:
torch.cat(inputs)

tensor([[ 0.8123,  0.4575, -0.9634],
        [ 0.3268, -0.0092,  0.8424],
        [-0.2896, -0.2440, -0.5013],
        [-0.4043,  0.1324,  0.2618],
        [ 0.7479, -0.7517, -0.0047]])

In [56]:
torch.cat(inputs).view(len(inputs), 1, -1)

tensor([[[ 0.8123,  0.4575, -0.9634]],

        [[ 0.3268, -0.0092,  0.8424]],

        [[-0.2896, -0.2440, -0.5013]],

        [[-0.4043,  0.1324,  0.2618]],

        [[ 0.7479, -0.7517, -0.0047]]])

In [58]:
torch.cat(inputs).view(1, 1, -1)

tensor([[[ 0.8123,  0.4575, -0.9634,  0.3268, -0.0092,  0.8424, -0.2896,
          -0.2440, -0.5013, -0.4043,  0.1324,  0.2618,  0.7479, -0.7517,
          -0.0047]]])

# Example: An LSTM for Part-of-Speech Tagging

In [8]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


training_data = [
    # Tags are: DET - determiner; NN - noun; V - verb
    # For example, the word "The" is a determiner
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]

word_to_ix = {}

In [9]:
# For each words-list (sentence) and tags-list in each tuple of training_data
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:  # word has not been assigned an index yet
            word_to_ix[word] = len(word_to_ix)  # Assign each word with a unique index
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}  # Assign each tag with a unique index

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [10]:
# Create the model:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [11]:
# Train the model
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

tensor([[-1.0477, -0.9759, -1.3004],
        [-1.0864, -0.9507, -1.2870],
        [-1.1187, -0.9845, -1.2051],
        [-1.1188, -0.9834, -1.2064],
        [-1.0595, -0.9589, -1.3091]])
tensor([[-0.0317, -3.8281, -4.6566],
        [-3.7377, -0.0485, -3.7476],
        [-3.1539, -3.4553, -0.0772],
        [-0.0592, -3.7359, -3.3930],
        [-4.4938, -0.0199, -4.7633]])


In [12]:
training_data[0][0]

['The', 'dog', 'ate', 'the', 'apple']

In [35]:
results = [(tag_scores == max(i)).nonzero(as_tuple=False)[0][1].item() for i in tag_scores]
results

[0, 1, 2, 0, 1]

In [37]:
ix_to_tag = {v:k for k,v in tag_to_ix.items()}
[ix_to_tag[i] for i in results]

['DET', 'NN', 'V', 'DET', 'NN']

# Word Embeddings in Pytorch
The module that allows you to use embeddings is torch.nn.Embedding, which takes two arguments: the vocabulary size, and the dimensionality of the embeddings.

To index into this table, you must use torch.LongTensor (since the indices are integers, not floats).



In [38]:
# Author: Robert Guthrie

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [39]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
print(lookup_tensor)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([0])
tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]],
       grad_fn=<EmbeddingBackward0>)


In [50]:
hello_embed.view((1, -1))

tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]],
       grad_fn=<ViewBackward0>)

# An Example: N-Gram Language Modeling

In [40]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10

# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.
# Each tuple is ([ word_i-CONTEXT_SIZE, ..., word_i-1 ], target word)
ngrams = [
    (
        [test_sentence[i - j - 1] for j in range(CONTEXT_SIZE)],
        test_sentence[i]
    )
    for i in range(CONTEXT_SIZE, len(test_sentence))
]
# Print the first 3, just so you can see what they look like.
print(ngrams[:3])


[(['forty', 'When'], 'winters'), (['winters', 'forty'], 'shall'), (['shall', 'winters'], 'besiege')]


In [45]:
vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}
print(word_to_ix)

{'Then': 0, "deserv'd": 1, 'field,': 2, 'couldst': 3, 'answer': 4, 'see': 5, 'being': 6, 'warm': 7, 'so': 8, 'beauty': 9, 'of': 10, "youth's": 11, 'cold.': 12, 'child': 13, 'shame,': 14, "'This": 15, 'old': 16, 'thou': 17, 'winters': 18, 'forty': 19, 'treasure': 20, 'his': 21, "beauty's": 22, 'deep': 23, 'livery': 24, 'If': 25, 'days;': 26, 'an': 27, 'thy': 28, 'and': 29, 'more': 30, 'say,': 31, 'in': 32, 'the': 33, 'it': 34, 'praise.': 35, 'make': 36, 'fair': 37, "feel'st": 38, 'When': 39, 'dig': 40, 'proud': 41, 'my': 42, 'succession': 43, "totter'd": 44, 'now,': 45, 'on': 46, 'weed': 47, 'where': 48, 'worth': 49, 'lies,': 50, 'brow,': 51, 'trenches': 52, 'gazed': 53, 'praise': 54, 'art': 55, 'be': 56, 'much': 57, 'own': 58, 'thriftless': 59, 'when': 60, 'mine': 61, 'Shall': 62, 'shall': 63, 'besiege': 64, 'within': 65, 'sum': 66, 'all-eating': 67, 'Proving': 68, 'Were': 69, 'made': 70, 'new': 71, 'were': 72, 'count,': 73, 'thine': 74, 'blood': 75, 'a': 76, 'small': 77, 'thine!': 78,

In [51]:
class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    for context, target in ngrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

[521.7581403255463, 519.2174344062805, 516.6933262348175, 514.1854174137115, 511.6928012371063, 509.2154755592346, 506.7507917881012, 504.2979989051819, 501.85662150382996, 499.4271500110626]


In [52]:
# To get the embedding of a particular word, e.g. "beauty"
print(model.embeddings.weight[word_to_ix["beauty"]])

tensor([-1.4779, -1.1335, -0.1012,  0.3425, -1.0703, -0.8744,  0.2047,  0.3053,
         0.5360, -0.4311], grad_fn=<SelectBackward0>)
